# Issue \#4
---

https://github.com/UoS-SNe/LSST_tools/issues/4

---

When simulating, the numbers of similated SNID's and SNNames don't match:

```
> output_df["snname"].value_counts()
SN2005bf    7834
SN1993J     1877
SN1994I      289
Name: snname, dtype: int64
``` 

and 
```
> output_df["snindex"].value_counts()
23    746
9     745
6     745
18    739
24    727
17    711
7     701
13    701
12    691
10    674
19    654
14    336
2     325
21    325
0     310
22    304
15    277
3      50
4      44
5      39
20     38
1      35
11     31
16     26
8      26
Name: snindex, dtype: int64
```
![snids_sdss_sesne_2](https://user-images.githubusercontent.com/10420166/27956030-b34cee24-630f-11e7-937e-808508f0fcb4.png)

---

## Current Implementation:

```
        ## Choose SN Type
        subtype = lsstt.sims.choose_subtype()
        if verbose: print(subtype)

        ## Choose SN
        w = np.where(info.table["Type"] == subtype)[0]
        snindex = np.random.choice(w)
        snname = pcc.utils.b(info.table["snname"].data[w][0])
        if verbose: print(w, snname)
```

In [19]:
%matplotlib inline

import os 
import json
import pandas as pd
import numpy as np

import pycoco as pcc
import lsst_tools as lsstt

In [7]:
verbose=True

Check infofile looks OK

In [12]:
info = pcc.InfoClass()
# info.load()
# use only good SNe
info.load(path = "/Users/berto/Code/CoCo/data/info/info_good.dat")

In [16]:
info.table.show_in_notebook()

idx,snname,z_obs,mu,RA,Dec,Type,SkyCoords
,,,,,,,"deg,deg"
0,SN1993J,-0.0001,27.82,09:55:24.7747,+69:01:13.702,IIb,"148.85322791666664,69.02047277777778"
1,SN1994I,0.0015,29.5,13:29:54.072,+47:11:30.50,Ic,"202.47529999999995,47.191805555555554"
2,SN1996cb,0.0024,30.69,11:03:41.98,+28:54:13.7,IIb,"165.92491666666666,28.903805555555554"
3,SN1998bw,0.0085,32.69,19:35:03.17,-52:50:46.1,Ic,"293.76320833333324,-52.846138888888895"
4,SN2002ap,0.0022,29.76,01:36:23.85,+15:45:13.2,Ic,"24.099375,15.753666666666666"
5,SN2003jd,0.0188,34.33,23:21:03.38,-04:53:45.5,Ic,"350.2640833333333,-4.8959722222222215"
6,SN2005bf,0.0189,34.73,10:23:56.99,-03:11:29.3,Ib,"155.9874583333333,-3.191472222222222"
7,SN2005hg,0.021,34.54,01:55:41.87,+46:47:47.4,Ib,"28.924458333333327,46.796499999999995"
8,SN2006aj,0.0334,35.9,03:21:39.670,+16:52:02.27,Ic,"50.41529166666666,16.867297222222223"


Infofile looks fine - must be in the sims

In [25]:
## Choose SN Type
subtype = lsstt.sims.choose_subtype()
if verbose: print(subtype)

IIb


In [29]:
np.where(info.table["Type"] == subtype)[0]

array([ 0,  2, 14, 15, 21, 22])

In [23]:
w = np.where(info.table["Type"] == subtype)[0]
snindex = np.random.choice(w)
snname = pcc.utils.b(info.table["snname"].data[w][0])
if verbose: print(w, snname, snindex)

[ 6  7  9 10 12 13 17 18 19 23 24] b'SN2005bf' 12


There it is. I can actually remember the edit that caused it. 
`w` was being assigned the array, as `numpy.where` returns a tuple, in line 1 above.
`snindex` was being assigned correctly, but `snname` used `w`, rather than the index. Corrected, this should look like:

In [37]:
w = np.where(info.table["Type"] == subtype)[0]
snindex = np.random.choice(w)
snname = pcc.utils.b(info.table["snname"].data[snindex])
if verbose: print(w, snname, snindex)

[ 0  2 14 15 21 22] b'SN2011ei' 22


---
# Fixed